# 2D SWFlow - Periodic waves over submerged bar
------------------------------------------------------------------------------------------

This notebook uses Proteus to reproduce the 1993/1994 experiments of Beji and Battjes to investigate the propagation of periodic waves over a submerged bar.

The original domain is defined to be D=[0, 37.3m]x[0,4m]. However, we introduce 6m (on the left) for wave generation and 12.7m (on the right) for wave absorption so the full computational domain is D=[-6, 50m]x[0,4m]. The topography (interchanged with the bathymetry nomenclature) is a trapezoidal profile simulating a sand bar. The periodic waves are generated on the left side of the domain and propagate to the right.

### References

- S. Beji and J. Battjes. Numerical simulation of nonlinear wave propagation over a bar. Coastal Engineering,         23(1):1 – 16, 1994. [https://doi.org/10.1016/0378-3839(94)90012-4](https://doi.org/10.1016/0378-3839(94)90012-4)

# Running the benchmark via the terminal

The `parun` script can be to execute the python script file: `reef_island_runup.py`. There are several argument that can be supplied to the `parun` script to define various runtime options. All available options are listed when executing `parun -h` in the command line. Common command-line options are as follows:

**Option** | **Description**
:---: | :---:
 -v   | Print logging information to standard output
 -O PETSCOPTIONSFILE  | Text file of options to pass to Petsc library
 -D DATADIR | Set data directory for output storage
 -l LOGLEVEL | Store runtime information at the log level, 0 = none, 10 = everything
 -b BATCHFILENAME | Text file of auxiliary commands to execute along with main program
 -G gatherArchive | Collect data files into single file at end of simulation (will require more computational resources on large runs)
 -H hotStart | Use the last step in the archive as the initial condition and continue appending to the archive
 --SWEs | To consider SWEs applications
 
 
To run the script on more than one rank, one can invoke the following: `mpiexec -n <number of cores>` before the use of `parun` in the command line. 

## Context options for run file

Most (if not all) Proteus run files `benchmark_name.py` (in this case `beji_periodic.py`) contain run time options specific to the model at hand. Here are some run time options for this particular example. For exact options, see the run file.

**Option** | **Description**
:---: | :---:
 sw_model | sw_model = {0,1} for {SWEs,DSWEs} 
 final_time  | Final time for simulation
 dt_output | Time interval to output solution
 mannings | Mannings roughness coefficient
 still_water_depth | Still water height above floor
 wave_period | Period of the waves 
 wave_height | Height of the waves
 
 
To modify the context options at run time, include the `-C` flag followed by `"option1=True option2=2 ..."`.

In [1]:
# Clean up previous data directory if it exists
!rm swflow_data/reef*

rm: cannot remove 'swflow_data/reef*': No such file or directory


In [3]:
# Then we run 
!PATH=/opt/proteus/linux/bin:$PATH mpiexec -np 2 parun --SWEs beji_periodic.py -v -l1 -C "final_time=25." -D run_data

[       0] Running Proteus version 1.8.0.dev0
Constructing GN_SW2DCV<CompKernelTemplate<2,4,3,3,3,3>());
Constructing GN_SW2DCV<CompKernelTemplate<2,4,3,3,3,3>());
2  nSpace_global
[       3] Setting initial conditions
[       4] Starting time stepping
[       4] ==============================================================
[       4] Solving over interval [ 0.00000e+00, 1.00000e-03]
[       4] ==============================================================
[       4] ==============================================================
[       4] Solving over interval [ 1.00000e-03, 1.00000e-01]
[       4] ==============================================================
[       7] ==============================================================
[       7] Solving over interval [ 1.00000e-01, 2.00000e-01]
[       7] ==============================================================
[       9] ==============================================================
[       9] Solving over interval [ 2.00000e-01

## Post-process the solution using ipygany

In [1]:
# Get dependencies
import sys
sys.path.append('/opt/proteus_visualization')
from hdf5_loader import extract_arrays_metadata, extract_array
import numpy as np
from ipywidgets import Image
from ipywidgets import Play, IntSlider, HBox, link
from ipygany import Scene, Data, Component, PolyMesh, Water, UnderWater, Data, Component, Threshold
from ipydatawidgets import NDArrayWidget

In [2]:
# Load our data
arrays_metadata = extract_arrays_metadata('./run_data/beji_periodic.h5')

mem_vertices = extract_array(arrays_metadata, 'nodesSpatial_Domain0')
vertices = np.array(mem_vertices[:, 0:2])

indices = extract_array(arrays_metadata, 'elementsSpatial_Domain0')

# This never changes, we extract it only once
bathymetry = extract_array(arrays_metadata, 'bathymetry0_t0')

# Get texture for topography
texture = Image.from_file('./cement.jpg')

In [3]:
# Define simulation parameters
warp_value = 20.
num_of_steps = 250

In [4]:
# Caching arrays on the front-end using NDArrayWidgets
h_cached = []
water_vertices_cached = []
for i in range(num_of_steps):
    h = extract_array(arrays_metadata, 'h_t{}'.format(i))

    z_water = h + bathymetry
    water_vertices = np.append(vertices, z_water.reshape((z_water.shape[0], 1)) * warp_value, axis=1).flatten()

    h_cached.append(NDArrayWidget(array=h))
    water_vertices_cached.append(NDArrayWidget(array=water_vertices))   

In [5]:
# Set up ipygany for visualizing the solution 

h_component = Component(name='h', array=h_cached[0])

water_mesh = PolyMesh(
    vertices=water_vertices_cached[0],
    triangle_indices=indices,
    data={'h': [h_component]}
)

actual_water = Threshold(water_mesh, input='h', min=1e-3, max=1000)

floor = PolyMesh(
    vertices=np.append(vertices, bathymetry.reshape((bathymetry.shape[0], 1)) * warp_value, axis=1),
    triangle_indices=indices,
    data={'underwater': [h_component]}
)

water = Water(
    actual_water, 
    under_water_blocks=(UnderWater(floor), ),
    caustics_enabled=True
)

scene = Scene((water, ))

def update_step(change):
    i = change['new']

    h_component.array = h_cached[i]
    water_mesh.vertices = water_vertices_cached[i]

play = Play(description='Step:', min=0, max=num_of_steps-1, value=0, interval=100)
play.observe(update_step, names=['value'])

progress = IntSlider(value=0, step=1, min=0, max=num_of_steps-1)
link((progress, 'value'), (play, 'value'))

display(HBox((play, progress)))

# Visualize solution 
scene

Scene(children=[Water(caustics_enabled=True, input=(), parent=Threshold(input='h', max=1000.0, min=0.001, pare…

In [6]:
# Define some visualization parameters
water.caustics_factor = 0.20
water.under_water_blocks[0].texture = texture
scene.background_color='aliceblue'